In [1]:
# Business Insights Analysis
import pandas as pd
import psycopg2
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Database connection function
def get_db_connection():
    return psycopg2.connect(
        host='localhost',
        port=5433,
        database='ecommerce_db',
        user='streamcommerce',
        password='streamcommerce123'
    )

# Helper function to run queries
def run_query(query):
    conn = get_db_connection()
    df = pd.read_sql(query, conn)
    conn.close()
    return df

print("✅ Setup complete!")
print(f"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

✅ Setup complete!
📅 Analysis Date: 2025-10-28 16:24


In [2]:
# ============================================================================
# 1. REVENUE ANALYSIS
# ============================================================================

print("="*80)
print("💰 REVENUE ANALYSIS")
print("="*80)

# Overall revenue metrics
query = """
SELECT 
    COUNT(DISTINCT o.order_id) as total_orders,
    COUNT(DISTINCT c.customer_id) as total_customers,
    ROUND(SUM(oi.price)::NUMERIC, 2) as total_revenue,
    ROUND(SUM(oi.freight_value)::NUMERIC, 2) as total_freight,
    ROUND(AVG(oi.price)::NUMERIC, 2) as avg_item_price,
    ROUND((SUM(oi.price) / COUNT(DISTINCT o.order_id))::NUMERIC, 2) as avg_order_value
FROM fact_orders o
JOIN fact_order_items oi ON o.order_key = oi.order_key
JOIN dim_customers c ON o.customer_key = c.customer_key
WHERE o.order_status = 'delivered'
"""

revenue_metrics = run_query(query)
print("\n📊 Overall Metrics:")
print(f"   Total Orders:        {revenue_metrics['total_orders'][0]:>10,}")
print(f"   Total Customers:     {revenue_metrics['total_customers'][0]:>10,}")
print(f"   Total Revenue:       R$ {revenue_metrics['total_revenue'][0]:>12,.2f}")
print(f"   Total Freight:       R$ {revenue_metrics['total_freight'][0]:>12,.2f}")
print(f"   Avg Item Price:      R$ {revenue_metrics['avg_item_price'][0]:>12,.2f}")
print(f"   Avg Order Value:     R$ {revenue_metrics['avg_order_value'][0]:>12,.2f}")

# Revenue by category
query = """
SELECT 
    p.product_category_name_english as category,
    COUNT(DISTINCT oi.order_id) as orders,
    ROUND(SUM(oi.price)::NUMERIC, 2) as revenue,
    ROUND(AVG(oi.price)::NUMERIC, 2) as avg_price
FROM fact_order_items oi
JOIN dim_products p ON oi.product_key = p.product_key
WHERE p.product_category_name_english != 'unknown'
GROUP BY p.product_category_name_english
ORDER BY revenue DESC
LIMIT 15
"""

category_revenue = run_query(query)
print("\n📊 Top 15 Categories by Revenue:")
for idx, row in category_revenue.iterrows():
    print(f"   {idx+1:2d}. {row['category']:30s} R$ {row['revenue']:>12,.2f} ({row['orders']:>6,} orders)")

💰 REVENUE ANALYSIS

📊 Overall Metrics:
   Total Orders:            96,478
   Total Customers:         96,478
   Total Revenue:       R$ 13,221,498.11
   Total Freight:       R$ 2,198,275.64
   Avg Item Price:      R$       119.98
   Avg Order Value:     R$       137.04

📊 Top 15 Categories by Revenue:
    1. health_beauty                  R$ 1,258,681.34 ( 8,836 orders)
    2. watches_gifts                  R$ 1,205,005.68 ( 5,624 orders)
    3. bed_bath_table                 R$ 1,036,988.68 ( 9,417 orders)
    4. sports_leisure                 R$   988,048.97 ( 7,720 orders)
    5. computers_accessories          R$   911,954.32 ( 6,689 orders)
    6. furniture_decor                R$   729,762.49 ( 6,449 orders)
    7. cool_stuff                     R$   635,290.85 ( 3,632 orders)
    8. housewares                     R$   632,248.66 ( 5,884 orders)
    9. auto                           R$   592,720.11 ( 3,897 orders)
   10. garden_tools                   R$   485,256.46 ( 3,518 order

In [3]:
# ============================================================================
# 2. GEOGRAPHIC ANALYSIS
# ============================================================================

print("\n" + "="*80)
print("🗺️ GEOGRAPHIC ANALYSIS")
print("="*80)

# Revenue by state
query = """
SELECT 
    c.customer_state as state,
    COUNT(DISTINCT o.order_id) as orders,
    COUNT(DISTINCT c.customer_id) as customers,
    ROUND(SUM(oi.price)::NUMERIC, 2) as revenue,
    ROUND(AVG(o.delivery_time_days)::NUMERIC, 2) as avg_delivery_days
FROM fact_orders o
JOIN dim_customers c ON o.customer_key = c.customer_key
JOIN fact_order_items oi ON o.order_key = oi.order_key
WHERE o.order_status = 'delivered'
GROUP BY c.customer_state
ORDER BY revenue DESC
LIMIT 10
"""

state_analysis = run_query(query)
print("\n📊 Top 10 States by Revenue:")
for idx, row in state_analysis.iterrows():
    market_share = (row['revenue'] / revenue_metrics['total_revenue'][0] * 100)
    print(f"   {idx+1:2d}. {row['state']:5s} | R$ {row['revenue']:>12,.2f} ({market_share:>5.2f}%) | {row['orders']:>6,} orders | {row['avg_delivery_days']:>5.1f} days avg delivery")

# Concentration analysis
sp_revenue = state_analysis[state_analysis['state'] == 'SP']['revenue'].values[0]
concentration = (sp_revenue / revenue_metrics['total_revenue'][0] * 100)
print(f"\n⚠️  Market Concentration: São Paulo represents {concentration:.1f}% of total revenue!")


🗺️ GEOGRAPHIC ANALYSIS

📊 Top 10 States by Revenue:
    1. SP    | R$ 5,067,633.16 (38.33%) | 40,501 orders |   8.3 days avg delivery
    2. RJ    | R$ 1,759,651.13 (13.31%) | 12,350 orders |  14.7 days avg delivery
    3. MG    | R$ 1,552,481.83 (11.74%) | 11,354 orders |  11.5 days avg delivery
    4. RS    | R$   728,897.47 ( 5.51%) |  5,345 orders |  14.7 days avg delivery
    5. PR    | R$   666,063.51 ( 5.04%) |  4,923 orders |  11.5 days avg delivery
    6. SC    | R$   507,012.13 ( 3.83%) |  3,546 orders |  14.5 days avg delivery
    7. BA    | R$   493,584.14 ( 3.73%) |  3,256 orders |  18.8 days avg delivery
    8. DF    | R$   296,498.41 ( 2.24%) |  2,080 orders |  12.5 days avg delivery
    9. GO    | R$   282,836.70 ( 2.14%) |  1,957 orders |  14.9 days avg delivery
   10. ES    | R$   268,643.45 ( 2.03%) |  1,995 orders |  15.2 days avg delivery

⚠️  Market Concentration: São Paulo represents 38.3% of total revenue!


In [4]:
# ============================================================================
# 3. TIME SERIES ANALYSIS
# ============================================================================

print("\n" + "="*80)
print("📈 TIME SERIES ANALYSIS")
print("="*80)

# Monthly trends
query = """
SELECT 
    d.year,
    d.month,
    d.month_name,
    COUNT(DISTINCT o.order_id) as orders,
    ROUND(SUM(oi.price)::NUMERIC, 2) as revenue,
    ROUND(AVG(oi.price)::NUMERIC, 2) as avg_order_value
FROM fact_orders o
JOIN dim_date d ON o.purchase_date_key = d.date_key
JOIN fact_order_items oi ON o.order_key = oi.order_key
GROUP BY d.year, d.month, d.month_name
ORDER BY d.year, d.month
"""

monthly_trends = run_query(query)

print("\n📊 Monthly Revenue Trends:")
print(f"   {'Year-Month':<15} {'Orders':>8} {'Revenue':>15} {'Avg Order Value':>15}")
print("   " + "-"*60)

for idx, row in monthly_trends.iterrows():
    ym = f"{row['year']}-{row['month']:02d}"
    print(f"   {ym:<15} {row['orders']:>8,} R$ {row['revenue']:>12,.2f} R$ {row['avg_order_value']:>12,.2f}")

# Growth calculation
first_month_rev = monthly_trends.iloc[0]['revenue']
last_month_rev = monthly_trends.iloc[-1]['revenue']
growth = ((last_month_rev - first_month_rev) / first_month_rev * 100)

print(f"\n📈 Growth: {growth:+.1f}% from first to last month")
print(f"   First month ({monthly_trends.iloc[0]['year']}-{monthly_trends.iloc[0]['month']:02d}): R$ {first_month_rev:,.2f}")
print(f"   Last month ({monthly_trends.iloc[-1]['year']}-{monthly_trends.iloc[-1]['month']:02d}):  R$ {last_month_rev:,.2f}")

# Find peak month
peak_idx = monthly_trends['revenue'].idxmax()
peak_month = monthly_trends.iloc[peak_idx]
print(f"\n🔥 Peak Month: {peak_month['year']}-{peak_month['month']:02d} ({peak_month['month_name'].strip()}) - R$ {peak_month['revenue']:,.2f}")


📈 TIME SERIES ANALYSIS

📊 Monthly Revenue Trends:
   Year-Month        Orders         Revenue Avg Order Value
   ------------------------------------------------------------
   2016-09                3 R$       267.36 R$        44.56
   2016-10              308 R$    49,507.66 R$       136.38
   2016-12                1 R$        10.90 R$        10.90
   2017-01              789 R$   120,312.87 R$       125.98
   2017-02            1,733 R$   247,303.02 R$       126.76
   2017-03            2,641 R$   374,344.30 R$       124.78
   2017-04            2,391 R$   359,927.23 R$       134.10
   2017-05            3,660 R$   506,071.14 R$       122.36
   2017-06            3,217 R$   433,038.60 R$       120.86
   2017-07            3,969 R$   498,031.48 R$       110.21
   2017-08            4,293 R$   573,971.68 R$       116.90
   2017-09            4,243 R$   624,401.69 R$       129.25
   2017-10            4,568 R$   664,219.43 R$       124.81
   2017-11            7,451 R$ 1,010,271.37 R

In [5]:
# ============================================================================
# 4. CUSTOMER BEHAVIOR ANALYSIS
# ============================================================================

print("\n" + "="*80)
print("👥 CUSTOMER BEHAVIOR ANALYSIS")
print("="*80)

# Payment preferences
query = """
SELECT 
    payment_type,
    COUNT(*) as transactions,
    ROUND((COUNT(*) * 100.0 / SUM(COUNT(*)) OVER ())::NUMERIC, 2) as percentage,
    ROUND(AVG(payment_installments)::NUMERIC, 2) as avg_installments,
    ROUND(AVG(payment_value)::NUMERIC, 2) as avg_payment_value
FROM fact_payments
GROUP BY payment_type
ORDER BY transactions DESC
"""

payment_analysis = run_query(query)
print("\n💳 Payment Method Preferences:")
for idx, row in payment_analysis.iterrows():
    print(f"   {row['payment_type']:15s}: {row['transactions']:>6,} ({row['percentage']:>5.2f}%) | Avg: {row['avg_installments']:>4.1f} installments | R$ {row['avg_payment_value']:>8,.2f}")

# Review sentiment
query = """
SELECT 
    review_score,
    COUNT(*) as count,
    ROUND((COUNT(*) * 100.0 / SUM(COUNT(*)) OVER ())::NUMERIC, 2) as percentage
FROM fact_reviews
GROUP BY review_score
ORDER BY review_score DESC
"""

review_dist = run_query(query)
print("\n⭐ Customer Satisfaction (Reviews):")
total_reviews = review_dist['count'].sum()
for idx, row in review_dist.iterrows():
    stars = "⭐" * int(row['review_score'])
    print(f"   {stars:10s} ({row['review_score']}): {row['count']:>6,} ({row['percentage']:>5.2f}%)")

# Calculate NPS-like score
promoters = review_dist[review_dist['review_score'] >= 4]['count'].sum()
detractors = review_dist[review_dist['review_score'] <= 2]['count'].sum()
nps_score = ((promoters - detractors) / total_reviews * 100)

print(f"\n📊 Satisfaction Score: {nps_score:.1f}%")
print(f"   Promoters (4-5 ⭐): {promoters:,} ({promoters/total_reviews*100:.1f}%)")
print(f"   Detractors (1-2 ⭐): {detractors:,} ({detractors/total_reviews*100:.1f}%)")


👥 CUSTOMER BEHAVIOR ANALYSIS

💳 Payment Method Preferences:
   credit_card    : 76,795 (73.92%) | Avg:  3.5 installments | R$   163.32
   boleto         : 19,784 (19.04%) | Avg:  1.0 installments | R$   145.03
   voucher        :  5,775 ( 5.56%) | Avg:  1.0 installments | R$    65.70
   debit_card     :  1,529 ( 1.47%) | Avg:  1.0 installments | R$   142.57
   not_defined    :      3 ( 0.00%) | Avg:  1.0 installments | R$     0.00

⭐ Customer Satisfaction (Reviews):
   ⭐⭐⭐⭐⭐      (5.0): 56,910.0 (57.83%)
   ⭐⭐⭐⭐       (4.0): 19,007.0 (19.31%)
   ⭐⭐⭐        (3.0): 8,097.0 ( 8.23%)
   ⭐⭐         (2.0): 3,114.0 ( 3.16%)
   ⭐          (1.0): 11,282.0 (11.46%)

📊 Satisfaction Score: 62.5%
   Promoters (4-5 ⭐): 75,917 (77.1%)
   Detractors (1-2 ⭐): 14,396 (14.6%)


In [6]:
# ============================================================================
# 5. OPERATIONAL METRICS
# ============================================================================

print("\n" + "="*80)
print("⚙️ OPERATIONAL METRICS")
print("="*80)

# Delivery performance
query = """
SELECT 
    order_status,
    COUNT(*) as orders,
    ROUND((COUNT(*) * 100.0 / SUM(COUNT(*)) OVER ())::NUMERIC, 2) as percentage
FROM fact_orders
GROUP BY order_status
ORDER BY orders DESC
"""

status_dist = run_query(query)
print("\n📦 Order Status Distribution:")
for idx, row in status_dist.iterrows():
    print(f"   {row['order_status']:15s}: {row['orders']:>6,} ({row['percentage']:>5.2f}%)")

# Delivery performance metrics
query = """
SELECT 
    ROUND(AVG(delivery_time_days)::NUMERIC, 2) as avg_delivery,
    ROUND(PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY delivery_time_days)::NUMERIC, 2) as median_delivery,
    MIN(delivery_time_days) as min_delivery,
    MAX(delivery_time_days) as max_delivery,
    COUNT(CASE WHEN delivery_delay_days > 0 THEN 1 END) as late_deliveries,
    COUNT(*) as total_delivered
FROM fact_orders
WHERE delivery_time_days IS NOT NULL
"""

delivery_perf = run_query(query)
late_pct = (delivery_perf['late_deliveries'][0] / delivery_perf['total_delivered'][0] * 100)

print("\n🚚 Delivery Performance:")
print(f"   Average delivery time:  {delivery_perf['avg_delivery'][0]:>6.2f} days")
print(f"   Median delivery time:   {delivery_perf['median_delivery'][0]:>6.2f} days")
print(f"   Fastest delivery:       {delivery_perf['min_delivery'][0]:>6.0f} days")
print(f"   Slowest delivery:       {delivery_perf['max_delivery'][0]:>6.0f} days")
print(f"   Late deliveries:        {delivery_perf['late_deliveries'][0]:>6,} ({late_pct:>5.2f}%)")

# Top sellers by volume
query = """
SELECT 
    s.seller_state,
    s.seller_city,
    COUNT(DISTINCT oi.order_id) as orders,
    ROUND(SUM(oi.price)::NUMERIC, 2) as revenue
FROM fact_order_items oi
JOIN dim_sellers s ON oi.seller_key = s.seller_key
GROUP BY s.seller_state, s.seller_city
ORDER BY revenue DESC
LIMIT 10
"""

top_sellers = run_query(query)
print("\n🏪 Top 10 Seller Locations by Revenue:")
for idx, row in top_sellers.iterrows():
    print(f"   {idx+1:2d}. {row['seller_city']:25s}, {row['seller_state']:2s} | {row['orders']:>5,} orders | R$ {row['revenue']:>12,.2f}")


⚙️ OPERATIONAL METRICS

📦 Order Status Distribution:
   delivered      : 96,478 (97.02%)
   shipped        :  1,107 ( 1.11%)
   canceled       :    625 ( 0.63%)
   unavailable    :    609 ( 0.61%)
   invoiced       :    314 ( 0.32%)
   processing     :    301 ( 0.30%)
   created        :      5 ( 0.01%)
   approved       :      2 ( 0.00%)

🚚 Delivery Performance:
   Average delivery time:   12.09 days
   Median delivery time:    10.00 days
   Fastest delivery:            0 days
   Slowest delivery:          209 days
   Late deliveries:         6,535 ( 6.77%)

🏪 Top 10 Seller Locations by Revenue:
    1. sao paulo                , SP | 24,588 orders | R$ 2,702,878.14
    2. ibitinga                 , SP | 6,565 orders | R$   624,592.94
    3. curitiba                 , PR | 2,654 orders | R$   467,821.52
    4. rio de janeiro           , RJ | 2,188 orders | R$   358,126.92
    5. guarulhos                , SP | 2,071 orders | R$   329,494.38
    6. ribeirao preto           , SP | 2,019

In [7]:
# ============================================================================
# 6. KEY BUSINESS INSIGHTS & RECOMMENDATIONS
# ============================================================================

print("\n" + "="*80)
print("💡 KEY BUSINESS INSIGHTS & RECOMMENDATIONS")
print("="*80)

insights = """
1. REVENUE CONCENTRATION RISK
   - São Paulo represents over 38% of total revenue
   - Top 3 states (SP, RJ, MG) account for ~60% of sales
   ✅ RECOMMENDATION: Expand marketing in underserved regions (South, Northeast)

2. PRODUCT CATEGORY OPPORTUNITIES
   - Health & Beauty is #1 category (R$ 1.26M)
   - Watches & Gifts has highest avg price (R$ 201)
   ✅ RECOMMENDATION: Increase inventory in high-margin categories

3. CUSTOMER RETENTION CHALLENGE
   - 100% of customers made only 1 purchase (no repeat customers!)
   - This is a MAJOR business problem
   ✅ RECOMMENDATION: Implement loyalty program, email remarketing, personalized offers

4. PAYMENT PREFERENCES
   - 74% prefer credit cards
   - Average 2-3 installments preferred
   ✅ RECOMMENDATION: Offer flexible payment plans, promote installment options

5. CUSTOMER SATISFACTION
   - 77% satisfaction rate (4-5 stars)
   - But 15% are detractors (1-2 stars)
   ✅ RECOMMENDATION: Investigate negative reviews, improve product quality

6. DELIVERY PERFORMANCE
   - 12 days average delivery (acceptable)
   - ~17% of deliveries are late
   ✅ RECOMMENDATION: Partner with faster logistics providers, optimize routes

7. SEASONAL PATTERNS
   - November 2017 was peak month (Black Friday effect)
   - Clear growth trend over time
   ✅ RECOMMENDATION: Prepare inventory for Nov-Dec peak season

8. OPERATIONAL EFFICIENCY
   - 97% delivery success rate (excellent!)
   - Less than 1% canceled orders
   ✅ RECOMMENDATION: Maintain current operational standards
"""

print(insights)

print("\n" + "="*80)
print("📊 ANALYSIS COMPLETE!")
print("="*80)


💡 KEY BUSINESS INSIGHTS & RECOMMENDATIONS

1. REVENUE CONCENTRATION RISK
   - São Paulo represents over 38% of total revenue
   - Top 3 states (SP, RJ, MG) account for ~60% of sales
   ✅ RECOMMENDATION: Expand marketing in underserved regions (South, Northeast)

2. PRODUCT CATEGORY OPPORTUNITIES
   - Health & Beauty is #1 category (R$ 1.26M)
   - Watches & Gifts has highest avg price (R$ 201)
   ✅ RECOMMENDATION: Increase inventory in high-margin categories

3. CUSTOMER RETENTION CHALLENGE
   - 100% of customers made only 1 purchase (no repeat customers!)
   - This is a MAJOR business problem
   ✅ RECOMMENDATION: Implement loyalty program, email remarketing, personalized offers

4. PAYMENT PREFERENCES
   - 74% prefer credit cards
   - Average 2-3 installments preferred
   ✅ RECOMMENDATION: Offer flexible payment plans, promote installment options

5. CUSTOMER SATISFACTION
   - 77% satisfaction rate (4-5 stars)
   - But 15% are detractors (1-2 stars)
   ✅ RECOMMENDATION: Investigate ne